<a href="https://colab.research.google.com/github/hdpark1208/StudyCode/blob/main/CryptoWorkoutWeek7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simplified DES 구현하기

## 2진수, 8진수, 16진수
2진수는 앞에 '0b'붙이면 됩니다.
또한 8진수, 16진수는 각각 '0o', '0x'를 붙이면 됩니다.

* 2진수: 0b
* 8진수: 0o
* 16진수: 0x

출처:[[파이썬] 2진수, 8진수, 16진수 다루기](https://www.daleseo.com/python-int-bases/)

In [ ]:
0b101010

42

또한 주어진 수를 2진수, 8진수, 16진수의 *문자열*로 변환할때는, `bin(), oct(), hex()`라는 명령어를 이용하면 됩니다.

In [ ]:
print(bin(42), oct(42), hex(42), bin(0b101010))

0b101010 0o52 0x2a 0b101010


### 다른 진수의 문자열을 숫자형으로 변환하기

bin(), oct(), hex()로 변환한 문자열들을 다시 숫자로 변환하기 위해서 자주 사용하시던 int() 함수에 진수의 base 값을 추가 인자로 넘겨주시면 됩니다. 두번째 인자에 아무 것도 입력하지 않으면 기본 설정인 10진법의 숫자로 변환합니다.



In [ ]:
print(int('0b101010', 2), int('0o52', 8), int('0x2a', 16))

42 42 42


format() 내장 함수를 이용하면 숫자를 다른 진수의 문자열로 바꿀 때 접두어를 제외할 수 있습니다.

In [ ]:
print(format(42, 'b'), format(42, 'o'), format(42, 'x'))

101010 52 2a


2진수, 8진수, 16진수 문자열을 10진수로 변환할때도 접두어를 생략해도 됩니다







In [ ]:
int('101010', 2) 

42

### 2진수의 연산
- `&` (Binary AND) : bit 단위 and연산
- `|` (Binary OR) : bit 단위 or연산
- `^` (Binary XOR) : bit 단위 xor연산
- `~` (Binary NOT) : bit 단위 not연산(1의 보수)
- `<<` (Binary left Shift) : bit 단위 왼쪽으로 비트단위 밀기 연산
- `>>` (Binary right Shift) : bit 단위 오른쪽으로 비트단위 밀기 연산

In [ ]:
a = 0b10101010
b = 0b01110011
print('a = ',  a, ":", bin(a))
print('b = ',  b, ":", bin(b))
print('a & b = ',  a & b, ":", bin(a & b))
print('a | b = ',  a | b, ":", bin(a | b))
print('a ^ b = ',  a ^ b, ":", bin(a ^ b))
print('~a = ',  ~a, ":", bin(~a))

a =  170 : 0b10101010
b =  115 : 0b1110011
a & b =  34 : 0b100010
a | b =  251 : 0b11111011
a ^ b =  217 : 0b11011001
~a =  -171 : -0b10101011


연산결과는 십진수가 되는 것에 유의.
또한 맨 앞자리가 0이되면 **자리수가 줄어드는 것**에 유의

In [ ]:
a = 0b101
b = 0b011
c = 0b111
print('a & b = ',  a & b, ":", bin(a & b))
print('a ^ c = ',  a ^ c, ":", bin(a ^ c))

a & b =  1 : 0b1
a ^ c =  2 : 0b10


## 확장자 $E(R)$ 만들기
편의상 이진수들을 list형으로 표현합니다.
확장자 함수 $E(R_{i-1})$는 6비트 (문자열) $R_{i-1}=𝑎_1 𝑎_2 𝑎_3 𝑎_4 𝑎_5 𝑎_6$를 8비트 $𝑎_1 𝑎_2 𝑎_4 𝑎_3 𝑎_4 𝑎_3 𝑎_5 𝑎_6$로 확장합니다. 이러한 함수 `Des_Expander(R)`을 작성해봅시다. 

In [ ]:
def Des_Expander(R):
  return R[:2]+R[3]+R[2:4]+R[2]+R[4:]

In [ ]:
R='011001'
Des_Expander(R)

'01010101'

## XOR 구현하기
이번에는 같은 길이의 (list 형) 두 값의 XOR를 반환하는 함수 `DES_XOR(A, B)`를 작성해봅시다

In [ ]:
def DES_XOR(A, B):
  m = max(len(A), len(B))
  C = format(int(A, 2)^int(B, 2), 'b') # 주의: 앞부분이 0이 되면 길이가 줄어들 수 있음
  while len(C)<m: # 길이 유지를 위해서 만약 앞부분이 0이 되어 없어지면 다시 '0'을 붙여줍니다.
    C = '0'+C
  return C

In [ ]:
DES_XOR('111', '101')

'010'

## S-Box 구현하기
아래의 S Box에서 4비트 문자열(list) $R=b_1 b_2 b_3 b_4$에 대해 이진수 $b_1$행, $b_2 b_3 b_4$열에  해당하는 값을 반환하는 함수 `DES_S1(R)`과 `DES_S2(R)`을 작성해봅시다. (각 행과 열은 0행, 0열부터 시작합니다.)
$
S_1: 
\begin{bmatrix}
101 & 010 & 001 & 110 & 011 & 100 & 111 & 000 \\
001 & 100 & 110 & 010 & 000 & 111 & 101 & 011
\end{bmatrix}
$
$
S_2: 
\begin{bmatrix}
100 & 000 & 110 & 101 & 111 & 001 & 011 & 010 \\
101 & 011 & 000 & 111 & 110 & 010 & 001 & 100
\end{bmatrix}
$

In [ ]:
def DES_S1(R):
  if len(R)!=4:
    print("4비트(4개의 0, 1로 이루어진 문자열)를 입력해야 합니다")
    return
  S1=[['101', '010', '001', '110', '011', '100', '111', '000'], \
      ['001', '100', '110', '010', '000', '111', '101', '011']]
  return S1[int(R[0])][int(R[1])*4+int(R[2])*2+int(R[3])]

def DES_S2(R):
  if len(R)!=4:
    print("4비트(4개의 0, 1로 이루어진 문자열)를 입력해야 합니다")
    return
  S2=[['100', '000', '110', '101', '111', '001', '011', '010'], \
      ['101', '011', '000', '111', '110', '010', '001', '100']]
  return S2[int(R[0])][int(R[1])*4+int(R[2])*2+int(R[3])]
  

In [ ]:
DES_S1('1010')

'110'

## $f(R_{i-1}, K_i)$ 구현하기
6비트 R, 8비트 K에 대해 강의에서 설명한 f함수의 역할을 하는 `DES_f(R, K)`를 구현해봅시다

In [ ]:
def DES_f(R, K):
  if len(R)!=6 or len(K)!=8:
    print("입력을 다시 확인해보세요!")
    return
  A = DES_XOR(Des_Expander(R), K)[:4]
  B = DES_XOR(Des_Expander(R), K)[4:]
  return DES_S1(A)+DES_S2(B)

In [ ]:
DES_f('100110', '01100101')

'000100'

## Simplified DES 완성하기
자, 이제 12비트의 메시지(평문) $L_0 R_0$과 9비트의 암호화키 $K$에 대해 $n$라운드의 Simplified DES 암호화를 하는 함수 `DES(M, K, n)`를 구현해봅시다.

In [ ]:
def DES(M, K, n):
  L, R = M[:6], M[6:]
  for i in range(n):
    K1 = K[i:]+K[:i]
    # print("The key for round {}:{}".format(i, K1))
    L, R = R, DES_XOR(L, DES_f(R, K1[:8]))
  return L+R
  

In [ ]:
DES('011100100110', '011001010',1)

'100110011000'

## 복호화
암호문의 좌우를 바꾸고 라운드별 암호화키를 역순으로 적용하면, 
메시지(평문)을 얻을 수 있습니다

In [ ]:
def DES_Decrypt(M, K, n):
  L, R = M[6:], M[:6]  # 먼저 좌우를 바꿉니다
  for i in reversed(range(n)):
    K1 = K[i:]+K[:i]
    # print("The key for round {}:{}".format(i, K1))
    L, R = R, DES_XOR(L, DES_f(R, K1[:8]))
  return R+L  # 다시 좌우를 바꿔서 원래의 메시지를 복구합니다.

In [ ]:
M = '011100100110'
K = '011001010'
C=DES(M, K,3)
print(C)

print(DES_Decrypt(C, K, 3))

000111001100
011100100110


## Diffusion과 Confusion 확인해보기

1. 메시지(평문)의 1비트(1자리)만 바꿔가면서 암호문의 어떤 자리들이 바뀌는지 확인해봅시다.
2. 암호화 키의 1비트(1자리)만 바꿔가면서 암호문의 어떤 자리들이 바뀌는지 확인해봅시다.

In [ ]:
M = '011100100110'
K = '011001010'
print(DES(M, K,6))
print(DES(DES_XOR(M, '100000000000'), K,6))
print(DES(DES_XOR(M, '010000000000'), K,6))
print(DES(DES_XOR(M, '001000000000'), K,6))
print(DES(DES_XOR(M, '000100000000'), K,6))
print(DES(DES_XOR(M, '000010000000'), K,6))
print(DES(DES_XOR(M, '000000000001'), K,6))

111111110101
000110000000
110001101011
110100110101
101011100111
001011101111
101100000110


In [ ]:
M = '011100100110'
K = '011001010'
print(DES(M, K,6))
print(DES(M, DES_XOR(K, '100000000'),6))
print(DES(M, DES_XOR(K, '010000000'),6))
print(DES(M, DES_XOR(K, '001000000'),6))
print(DES(M, DES_XOR(K, '000100000'),6))

111111110101
001011110000
101011001010
101001001100
000000000111


In [ ]:
for n in range(1, 4):
  print(DES('011100100110', '011000101',n))

100110011001
011001000010
000010010100


## Simplified DES 공격하기

주어진 평문과 암호문을 가지고 공격하는 것을 생각해봅시다.


In [ ]:
M = '011100100110' # 먼저 테스트를 위한 암호문을 하나 만들어봅시다
K = '011001010'
N = 4
print(DES(M, K, N))

001100010110


### 실습문제

1. Brute-force attack, 즉, 모든 경우의 암호화키를 가지고 찾는 코드를 작성해봅시다.
2. 위의 코드를 이용하여 평문 `M = '011100100110'`와 (라운드 4) Simplified DES를 적용한 암호문 `C = '001100010110'`에 사용한 암호화키를 찾아봅시다.

In [ ]:
M = '011100100110'
C = '001100010110'
N = 4
for i in range(2**9):
  K = format(i, 'b')
  while len(K)<9:
    K = '0'+K
  if DES(M, K, N)==C:
    print(K)
    break

011001010


## 실습문제

키 $K$를 사용한 4라운드 DES 암호화를 $E_K(M)$라 하자.
*Weak key*라 하면 모든 메시지 $M$에 대해 $E_K(E_K(M))=M$인 암호화키 $K$를 말한다. 

1. 이 단순화한 4라운드 DES에는 Weak key가 없음을 보여라. 
(즉, 각 $K$마다, $E_K(E_K(M))\ne M$인 메시지 $M$을 찾아야 한다)

2. 3라운드 DES의 경우에도 Weak key가 없는지 확인해보아라.


In [ ]:
N = 4

for i in range(2**9):
  K = format(i, 'b')
  while len(K)<9:
    K = '0'+K
  for j in range(2**12):  # 아래 weak key 아니라고 확인하는 과정을 깔끔하게!
    print(i,j)
    M = format(j, 'b')
    while len(M)<12:
      M = '0'+M
    M1 = DES(DES(M, K, N), K, N)
    if M1!=M:
      print("For key {}: Message {} is not same with {}".format(K, M, M1))
      break


0 0
For key 000000000: Message 000000000000 is not same with 011011100001
1 0
For key 000000001: Message 000000000000 is not same with 000110001111
2 0
For key 000000010: Message 000000000000 is not same with 001000010000
3 0
For key 000000011: Message 000000000000 is not same with 001010011110
4 0
For key 000000100: Message 000000000000 is not same with 000101101010
5 0
For key 000000101: Message 000000000000 is not same with 101011111101
6 0
For key 000000110: Message 000000000000 is not same with 000101010001
7 0
For key 000000111: Message 000000000000 is not same with 110101111111
8 0
For key 000001000: Message 000000000000 is not same with 001101011110
9 0
For key 000001001: Message 000000000000 is not same with 001011011100
10 0
For key 000001010: Message 000000000000 is not same with 101100000000
11 0
For key 000001011: Message 000000000000 is not same with 110011010110
12 0
For key 000001100: Message 000000000000 is not same with 110111001100
13 0
For key 000001101: Message 000

## Differential Cryptanalysis

암호화키는 모르지만, 원하는(chosen) 평문과 그 암호문을 볼 수 있는 상황을 가정하고, Simplified-DES를 3라운드만 돌린다고 가정한다. 

### 1 Round
메시지(평문)이 $L_1 R_1$이고 사용된 암호화키가 $K_2$라고 할때,
$𝑅_2⊕𝐿_1=𝑆𝐵𝑜𝑥(𝐸(𝑅_1 )⊕𝐾_2)$를 이용해서 키를 알아내봅시다.

`index()`라는 명령어를 사용하여 list에서 원소의 위치를 알아낼 수 있습니다.
예시) `[1,2,3].index(2)`

In [ ]:
M = '0'*12
K = '010011010' # 이 값을 모른다고 가정합니다.
C = DES(M, K, 1)
# print(C)
def DES_Attack_1Rd(M, C):
  L1, R1 = M[:6], M[6:]
  L2, R2 = C[:6], C[6:]
  L = [] # 키의 좌측 후보들
  R = [] # 키의 우측 후보들

  S1=[['101', '010', '001', '110', '011', '100', '111', '000'], \
      ['001', '100', '110', '010', '000', '111', '101', '011']]
  S2=[['100', '000', '110', '101', '111', '001', '011', '010'], \
      ['101', '011', '000', '111', '110', '010', '001', '100']]

  v1, v2 = DES_XOR(R2, L1)[:3], DES_XOR(R2, L1)[3:]

  for i in range(2): # 길이가 짧아지는 경우에 대해 대비 필요
    l1, l2  = format(S1[i].index(v1), 'b'), format(S2[i].index(v2), 'b')
    while len(l1)<3:
      l1 = '0'+l1
    while len(l2)<3:
      l2 = '0'+l2
  # print(l1, l2)
    L.append(DES_XOR(Des_Expander(R1)[:4], format(i, 'b')+l1))
    R.append(DES_XOR(Des_Expander(R1)[4:], format(i, 'b')+l2))
  print("Left: {}, Right: {}".format(L, R))



In [ ]:
M = '0'*12
K = '010011010' # 이 값을 모른다고 가정합니다.
C = DES(M, K, 1)

DES_Attack_1Rd(M, C)

M = '1'*12  # 이번에는 다른 메시지(평문)과 암호문을 가지고 K의 가능성을 좁혀봅시다
K = '010011010' # 이 값을 모른다고 가정합니다.
C = DES(M, K, 1)

DES_Attack_1Rd(M, C)

Left: ['0100', '1111'], Right: ['0111', '1101']
Left: ['1110', '0100'], Right: ['1101', '0011']


## Differential Cryptanalysis–3 Round 공략

1. 두 S-Box 입력의 XOR이 $𝐸(𝐿_4')$ 이고 출력의 XOR: $𝑅_4'⊕𝐿_1'$인 경우들 탐색
2. $(𝐸(𝐿_4 )⊕𝐾_4,𝐸(𝐿_4^∗ )⊕𝐾_4 )$ 이 위에 해당됨
3. $𝐾_4$의 가능한 경우들 파악
4. 딱 한가지 경우로 축소되도록 위의 과정 반복





In [ ]:
def DES_Attack_3Rd(M1, C1, M2, C2):
  L11, R11 = M1[:6], M1[6:]
  L21, R21 = M2[:6], M2[6:]
  DL1 = DES_XOR(L11, L21) # L1, L1*의 XOR
  DR1 = DES_XOR(R11, R21) # R1, R1*의 XOR

  L14, R14 = C1[:6], C1[6:]
  L24, R24 = C2[:6], C2[6:]
  DL4 = DES_XOR(L14, L24) # L4, L4*의 XOR
  DR4 = DES_XOR(R14, R24) # R4, R4*의 XOR

  Input_XOR1, Input_XOR2  = Des_Expander(DL4)[:4], Des_Expander(DL4)[4:]
  Output_XOR1, Output_XOR2  = DES_XOR(DR4, DL1)[:3], DES_XOR(DR4, DL1)[3:]

  print("Candidates for the left of Key:")
  for i in range(2**4):
    l = format(i, 'b')
    while len(l)<4:
      l = '0'+l
    if DES_XOR(DES_S1(l), DES_S1(DES_XOR(l, Input_XOR1))) == Output_XOR1:
      print(DES_XOR(l, Des_Expander(L14)[:4]))

  print("Candidates for the right of Key:")
  for i in range(2**4):
    l = format(i, 'b')
    while len(l)<4:
      l = '0'+l
    if DES_XOR(DES_S2(l), DES_S2(DES_XOR(l, Input_XOR2))) == Output_XOR2:
      print(DES_XOR(l, Des_Expander(L14)[4:]))

In [ ]:
M1 = '000111011011'
M2 = '101110011011'
K = '010011010' # 교재에서는 2라운드부터 시작하므로 미리 키를 1칸 shift해줍니다

C1 = DES(M1, K, 3)
C2 = DES(M2, K, 3)

# print(C1, C2)

DES_Attack_3Rd(M1, C1, M2, C2)

M3 = '010111011011'
C3 = DES(M3, K, 3)
DES_Attack_3Rd(M3, C3, M2, C2)

Candidates for the left of Key:
0011
1001
Candidates for the right of Key:
0100
1111
Candidates for the left of Key:
0011
1000
Candidates for the right of Key:
0100
1011


이제 딱 한자리를 모르므로, '0', '1' 둘 다 가정해서, 주어진 평문으로부터 원하는 암호문이 나오는 경우를 찾아봅니다.

In [ ]:
K4 = '0011'+'0100'

K = K4+'0' # 빈 한자리를 0이라고 가정
print(C1, DES(M1, K[7:]+K[:7], 3)) # K2부터 시작한 것에 주의

K = K4+'1' # 빈 한자리를 1이라고 가정
print(C1, DES(M1, K[7:]+K[:7], 3))

000011100101 001011101010
000011100101 000011100101


두번째 경우에 암호문이 일치하므로 `K[7:]+K[:7]`이 구하는 암호화키입니다.

In [ ]:
K[7:]+K[:7]

'010011010'

## 실습문제 
CBC mode를 구현하고 14번째 자리만 48비트의 두 평문의 암호문을 비교해보아라.
 - 긴 메시지(평문): $P=[P_1, P_2, ..., P_L]$
 - 암호문: $C=[C_1, C_2, ..., C_L]$
 - $𝐶_𝑗=𝐸_𝐾 (𝑃_𝑗⊕𝐶_{𝑗−1} )$
 - $𝑃_𝑗=𝐷_𝐾 (𝐶_𝑗 )⊕𝐶_{𝑗−1}$

In [ ]:
import random

P='' # 샘플 메시지와 암호화키를 random함수를 이용해서 만들어봅시다.
for i in range(48):
  if random.random()<0.5:
    P = P+'0'
  else:
    P = P+'1'
print(P)

K = ''
for i in range(9):
  if random.random()<0.5:
    K = K+'0'
  else:
    K = K+'1'
print(K)  

101010110111000101001001100010100011101000101011
100101110


In [ ]:
C0 = '101010101010'
C=''
for i in range(len(P)//12):
  C0 = DES(DES_XOR(P[12*i:12*(i+1)], C0), K, 4)
  C = C+C0

print(C)


010100110011110101101000000000011000110110000111


In [ ]:
C0 = '101010101010'
C=''
P = DES_XOR(P, '1'+'0'*13)

for i in range(len(P)//12):
  C0 = DES(DES_XOR(P[12*i:12*(i+1)], C0), K, 4)
  C = C+C0

print(C)

010100110011110101101000100100011000010001111111
